In [1]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

from sklearn import datasets, svm
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

RandomForestClassifier()

In [4]:
tensor_entrada = [('float_input', FloatTensorType([None,4]))]

In [5]:
modelo_onnx = convert_sklearn(clr, initial_types=tensor_entrada)
with open("modeloiris.onnx", "wb") as arquivo:
    arquivo.write(modelo_onnx.SerializeToString())

In [6]:
import onnxruntime as rt
sessao = rt.InferenceSession("modeloiris.onnx")
input_name = sessao.get_inputs()[0].name
label_name = sessao.get_outputs()[0].name
print(label_name)

output_label


In [7]:
predicoes = sessao.run([label_name], {input_name: X_test.astype(np.float32)})[0]
print(predicoes)

[2 2 0 0 0 2 2 0 0 2 1 1 1 2 2 0 2 1 0 2 0 2 0 0 2 1 0 0 2 1 1 1 0 2 1 0 0
 2]


In [9]:
test_predicao = np.array([[4.0,4.0,4.0,4.0]], dtype = np.float32)
resultado_predicao_test = sessao.run([label_name], {input_name: test_predicao})[0]
print(resultado_predicao_test)

[2]


In [10]:
if resultado_predicao_test[0]==0:
    prediction = "versicolor"
elif resultado_predicao_test[0]==1:
    prediction = "virginica"
else:
    prediction = "setosa"
    
print(prediction)

setosa
